In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from six.moves import cPickle 
import time
import scipy.spatial.distance as sp

In [2]:
def load_cifar10(dataset_number):

    for i in range(dataset_number):
        filename = '/home/alvin/notebook_home/ai_lab_image/corpus/cifar-10-batches-py/data_batch_' +str(i+1)
        f = open(filename, 'rb')
        datadict = cPickle.load(f,encoding='latin1')
        f.close()
    
        DX= datadict["data"] # 1000 x (32x32x3)
        DY= datadict['labels']
        #X = DX.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
        X[i*10000:(i+1)*10000] = np.array(DX)
        Y[i*10000:(i+1)*10000] = np.array(DY)
        #break
    return X,Y

In [ ]:
def dataset_crossvalidate()

In [3]:
def l1_dist(input_vec, train_set):
    #l1_formula = lambda x: sp.cdist(input_vec,x,'city_block')
    #l1_dist_vec= l1_formula(train_set)
    l1_dist_vec = sp.cdist(train_set,[input_vec],'cityblock')
    return l1_dist_vec

def l2_dist(input_vec, train_set):
    l2_dist_vec = sp.cdist(train_set,[input_vec],'euclidean')
    return l2_dist_vec

In [4]:
cifar_label = { '0':'airplane',
                '1':'automobile',
                '2':'bird',
                '3':'cat',
                '4':'deer',
                '5':'dog',
                '6':'frog',
                '7':'horse',
                '8':'ship',
                '9':'truck'}

In [ ]:
# show each class sample number on training data
print('dataset count = ' + str(len(X)))
for i in range(10):
    print('number of class '+ str(i) + ': ' + str(Y[np.where(Y==i)].size))

In [ ]:
#[option] show label information
Y[10000:10010]

In [ ]:
#[option] display cifar image with color
#Visualizing CIFAR 10 method 2
X3 = X.reshape(10000*dataset_number, 3, 32, 32).transpose(0,2,3,1)
pic_dim = 6
fig, axes1 = plt.subplots(pic_dim,pic_dim,figsize=(6,6))
for i in range(pic_dim*pic_dim):
    #print('i/pic_dim: ' + str(i/pic_dim) + 'i%pic_dim: ' + str(i%pic_dim))
    axes1[int(i/pic_dim)][int(i%pic_dim)].set_axis_off()
    axes1[int(i/pic_dim)][int(i%pic_dim)].imshow(X3[i])
    axes1[int(i/pic_dim)][int(i%pic_dim)].annotate(cifar_label[str(Y[i])],xy=(0.2,0.6))

In [ ]:
#[option] display cifar image with gray
#Visualizing CIFAR 10 method 2
XG = X.reshape(10000*dataset_number, 3, 1024).transpose(0,2,1).astype("float64") #1000 x 3 x 1024 --> 1000 x 1024 x 3
XG = np.mean(XG,axis=2)                                         #1000 x 1024
XG = XG.reshape(10000*dataset_number, 32, 32)
pic_dim = 6
fig, axes1 = plt.subplots(pic_dim,pic_dim,figsize=(6,6))
for i in range(pic_dim*pic_dim):
    #print('i/pic_dim: ' + str(i/pic_dim) + 'i%pic_dim: ' + str(i%pic_dim))
    axes1[int(i/pic_dim)][int(i%pic_dim)].set_axis_off()
    axes1[int(i/pic_dim)][int(i%pic_dim)].imshow(XG[i], cmap='gray')
    axes1[int(i/pic_dim)][int(i%pic_dim)].annotate(cifar_label[str(Y[i])],xy=(0.2,0.6))

In [ ]:
###############################################################
#   knn main code
###############################################################

In [5]:
###################################
# allocate space and load dataset
###################################
dataset_number = 5
X = np.zeros(shape=(dataset_number*10000,32*32*3),dtype='float64')
Y = np.empty([dataset_number * 10000], dtype='uint8')
X,Y = load_cifar10(dataset_number)

In [6]:
#####################################################
# change color to gray
#####################################################
XG = X.reshape(10000*dataset_number, 3, 1024).transpose(0,2,1).astype("float64") #10000 x 3 x 1024 --> 10000 x 1024 x 3
XG = np.mean(XG,axis=2)  #10000 x 1024

In [8]:
#####################################################
# assign traing set and testing set and run cross validation
#####################################################

t_rate   = 0.8
test_set_num  = int(round(len(X)*(1-t_rate),0))
for iter_num in range(1):
    test_index  = np.arange(iter_num*test_set_num, (iter_num+1)*test_set_num, 1)
    train_set   = np.delete(XG,test_index,0)
    train_label = np.delete(Y,test_index,0)
    test_set    = np.array(XG[test_index])
    test_label  = np.array(Y[test_index])
    
    #####################################################
    # run KNN algorithm
    #####################################################

    Stime = time.time()
    k = 1
    correct_num = 0
    pred = np.zeros(shape=(len(test_set),1),dtype='uint8')
    for i in range(len(test_set)):
        test_sample = test_set[i]
        dict_vec    = l1_dist(test_sample, train_set)
        #print(dict_vec)
        idx         = np.argpartition(dict_vec.reshape((-1,)),k)[0:k]
        vote        = train_label[idx[0:k]]
        pred[i]     = np.argmax(np.bincount(vote))
        #if i > 2: break
        #print(str(test_label[i]) + ' , ' + str(vote) + ' --> ' + str(pred[i]) + '\t progress ' + str(i) + '/' + str(len(test_set)))
    
        if test_label[i] == train_label[idx[0]]:
            correct_num += 1
    Etime = time.time()
    accuracy = correct_num/len(test_set)
    print('knn accuarcy is ' + str(float("{0:.3f}".format(accuracy*100)))+'%')
    print('exectution time knn w/ k = ' +str(k)+ ' is ' + str(float(("{0:.2f}".format(Etime-Stime)))) + 's')    

knn accuarcy is 0.0%
exectution time knn w/ k = 1 is 0.11s


In [ ]:
#dataset_number = 2 
#20000, testing sample 4000
'''
knn accuarcy is 28.775%
exectution time knn w/ k = 1 is 96.42s
knn accuarcy is 27.825%
exectution time knn w/ k = 1 is 95.66s
knn accuarcy is 27.6%
exectution time knn w/ k = 1 is 95.76s
knn accuarcy is 28.475%
exectution time knn w/ k = 1 is 95.18s
knn accuarcy is 29.225%
exectution time knn w/ k = 1 is 94.74s
'''

In [ ]:
dict_vec.reshape((-1,))

In [ ]:
# X uint8
# knn gray with k=3, 20,000[16,000, 4000],  dtype=uint8  = 10.578%
# knn gray with k=3, 30,000[24,000, 6000],  dtype=uint8  = 10.502% (1827.51s->30.45m)

# X float64
# knn gray with k=3, 10,000[ 8,000, 2000], dtype=float64 = 11.056% (214.57s-> 3.57m)
# knn gray with k=3, 20,000[16,000, 4000], dtype=float64 = 10.578% (815.93s->13.59m)

In [ ]:
k= 1
test_sample = test_set[0]
l1_dict_vec = l1_dist(test_sample,train_set)
idx = np.argpartition(l1_dict_vec,k)
#
print(Y[train_set_num+0])
print(Y[train_set_num+idx[0]])

In [ ]:
int_v = test_set[0]/255
trn_v = train_set[0]/255
print(int_v)
print(trn_v)

In [ ]:
print('np.linalg.norm = ' + str(np.linalg.norm((int_v - trn_v), ord=2)))
dist_l1 = sum(abs(int_v - trn_v))
print('sum(abs(a-b)) = ' + str(dist_l1))


In [ ]:
int_v = np.array([1,1,1,1,1])
tring_set = np.array([[2,2,2,2,2],[3,3,3,3,3],[4,4,4,4,4],[5,5,5,5,5]])
#dist_lambda = lambda x: np.linalg.norm((int_v - x), ord=1)
dist_lambda = lambda x: sum(abs(int_v - x))
#dist_vec = map{lamda x: np.linalg.norm((int_v - x), ord=1),train_set}
dist_vec = dist_lambda(tring_set)
dist_vec

In [ ]:
a = np.array([[1,0,1,0,1]])
b = np.array([[2,2,2,2,2],[3,3,3,3,3]])

In [ ]:
l1_dist_vec = sp.cdist(b,a,'cityblock')
l1_dist_vec

In [ ]:
a = np.array([1,2,3,4,5])
a

In [ ]:
b = np.delete([a],[1,3],1)
b

In [ ]:
t_rate   = 0.8
test_set_num  = int(round((len(X)*(1-t_rate)),0))
iter_number = 2
test_index = np.arange(iter_number*test_set_num, (iter_number+1)*test_set_num, 1)
train_set   = np.delete(XG,test_index,0)
train_lable = np.delete(Y,test_index,0)
test_set    = np.array(XG[test_index])
test_label  = np.array(Y[test_index])

In [ ]:
train_set.shape

In [ ]:
np.delete(np.linspace(1,10,10,dtype='uint8'),np.linspace(5,8,4,dtype='uint8'),0)

In [ ]:
train_set.shape

In [ ]:
test_index

In [ ]:
test_index = np.arange(iter_num*test_set_num, (iter_number+1)*test_set_num, 1)
test_index

In [ ]:
iter_num*test_set_num

In [ ]:
test_index[1600]